In [1]:
#k nearest neighbour
x = [[0],[1],[2],[3]]
y = [0, 0, 1, 1]
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x,y) #training the data
print(neigh.predict([[1.1]]))

[0]


In [2]:
print(neigh.predict_proba([[0.9]]))

[[0.66666667 0.33333333]]


In [3]:
# this whole code snippet for visualization the knn properly
from matplotlib.colors import ListedColormap
def knn_comparison(data,n_neighbors=15): #default n_neighbors is 15
    #This function finds k-NN and plots the data
    x = data[:,:2] #points
    y = data[:,2] #class labels
    #grid cell size
    h = 0.2
    cmap_light = ListedColormap('#FFAAAA','#AAAAFF') #set colors
    cmap_bold = ListedColormap('#FF0000','#0000FF')
    
    # the core classifier : k-NN
    clf = neighbors.KNeighborsClassifier(n_neighbors) 
    clf.fit(x,y) #train the data
    x_min, x_max = x[:,0].min()-1 , x[:,0].max()+1 #here create minimum and maximum values of x and y
    y_min, y_max = x[:,1].min()-1, y[:,0].max()+1
    #we create a mesh grid (x_min,y_min) to (x_max ,y_max) with 0.02 grid space
    xx,yy = np.meshgrid(np.arrange(x_min,x_max,h), np.arrange(y_min,y_max,h)) # meshgrid creates a matrix from the range x_min to x_max and y_min to y_max
    #we predict the value (either 0 or 1) of each element in the grid
    z = clf.predict(np.c_[xx.ravel(),yy.ravel()])
    #xx.ravel() will give a flatten array
    #np.c_: Translates slice objects to concatenation along the second axis
    #np.c_[np.array([1,2,3,4]), np.array([4,5,6,7])]
    #array([[1,4],[2,5],[3,6],[4,7]])
    # convert the out back to the xx shape
    x = z.reshape(xx.shape)
    #pcolormesh will plot the (xx,yy) grid with colors according to the values
    #it looks like decision boundary
    plt.figure()
    plt.pcolormesh(xx,yy,z,cmap=cmap_light)
    
    #scatter plot of with given plots
    plt.scatter(x[:,0],x[:,1],c=y,cmap=cmap_bold)
    #definig scale on both axises
    plt.xlim(xx.min(),xx.max())
    plt.ylim(yy.min(),yy.max())
    #set the title
    plt.title('K value = '+str(n_neighbors))
    plt.show()

In [ ]:
data = np.genformtext('overlap.csv',delimiter=',')
knn_comparison(data,1)
knn_comparison(data,5)

In [ ]:
# Part 1, data preprocessing
#define column names
names = ['x','y','class']
#loading training data
df = pd.read_csv('concertriccir2.csv',header=None,names=name)
print(df.head())

# create design matrix X and target vector Y
x = np.array(df.iloc[:,0:4])
y = np.array(df['class'])
# split the data set into train and test 
x_1,x_test,y_1,y_test = cross_validation.train_test_split(x,y,test_size=0.3,random_state=42) # firstly we divide the dataset to x_1 and x_test,wher x_1 belongs to x_train and x_cv,only 30% data is using for test
# split the train data set into cross validation train and cross validation test
x_tr,x_cv,y_tr,y_cv = cross_validation.train_test_split(x_1,y_1,test_size=0.3) # now we divide the x_1 data to x_train and x_cv data set,only 30% data is using for cv

for i in range(1,30,2):
    knn = KNeighborsClassifier(n_neighbors=i) # take the neighbour is 1,3,5...29 etc
    #fitting the model on crossvalidation train
    knn.fit(x_tr,y_tr)
    
    #predict the response on the corssvalidation train
    pred = knn.predict(x_cv)
    
    #evaluate cross validation accuracy, 
    acc = accuracy_score(y_cv,pred,normalize=True) * float(100) # 'y_cv' means orignal label of cross validation data,'pred' is predicted value of x_cv
    print('\nCV accuracy for k = %d is %d%%' % (i,acc))
knn = KNeighborsClassifier(1) #here for test accuracy when k=1
knn.fit(x_tr,y_tr) # fit the train data
pred = knn.predict(x_test) # predict the test accuracy by x_test when k=1
acc = accuracy_score(y_test,pred,normalize=True) * float(100)
print('\n**Test accuracy for k=1 is %d%%' % (acc))


In [4]:
# 10 fold cross validation
# creating odd list of K for KNN
myList = list(range(0,50))
neighbors = list(filter(lambda x: x %2 !=0,myList)) # the value stores in neighbors list is: 1,3,5,7...49

#empty list what will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn,x_train,y_train,cv=10,scoring='accuracy') # applygin 10 fold cross validation and find out the scores
    cv_scores.append(scores.mean()) # we take 10 values in scores using 10 fold cross validation,and we find out the mean value and append this value
    
#changing to misclassification error
MSE = [1-x for x in cv_scores] # cv_scores is the value which measure the accuracy,so if we subtract the value from 1 then we find the error,which one store in MSE list
# 0th index MSE value when k=1,1th index MSE value when k=3 etc
#determining the best k
optimal_k = neighbors[MSE.index(min(MSE))] #find out which one is minimum error, we take the min error value and find out the index number of 'MSE' list,whihc has same index number of 'neighbors' list(optimal k) 
print('\nThe optimal number of neighbors is %d.' %optimal_k)

#plot misclassification error vs K
plt.plot(neighbors,MSE)

for xy in zip(neighbors,np.round(MSE,3)):
    plt.annotate('(%s, %s)' %xy, xy=xy,textcoords='data')

plt.xlabel('Number of Neighbors K')
plt.ylable('Misclassification Error')
plt.show()
print("The misclassification error for each k value is : ",np.round(MSE,3))

In [ ]:
#find out optimal k
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k) # use the optimal_k as k
#fitting the model
knn_optimal.fit(x_train,y_train)

#predict the response
pred = knn_optimal.predict(x_test)

#evaluate accuracy
acc = accuracy_score(y_test,pred)*100
print('\nThe accuracy of the knn classifier for k=%d is %f%%' % (optimal_k))

In [3]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
X = [[-1.1],[0.2],[101.1],[0.3]]
clf = LocalOutlierFactor(n_neighbors=2)
clf.fit_predict(X)

G:\Anaconda installation\lib\site-packages\sklearn\neighbors\lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


array([ 1,  1, -1,  1])

In [ ]:
clf.negative_outlier_factor

In [20]:
#naive bayes
import numpy as np
X = np.random.randint(2,size=(6,100)) # randomly choosen value 0 or 1 which one is 6 rows and 100 coloumns
Y = np.array([1,2,3,4,4,5]) # predicted theses values(outputs)
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X, Y)
clf.predict(X[0:2]) 


array([1, 2])